Finding matching park with taxi zone

In [1]:
import geopandas as gpd
import pandas as pd
import folium 
import matplotlib.pyplot as plt
import mapclassify

manhattan_parks_gdf = gpd.read_file("manhattan_parks_geocoded_full.geojson")
#manhattan_taxi_zones_gdf = gpd.read_file("manhattan_taxi_zones.geojson")


In [2]:
manhattan_parks_gdf.columns

Index(['id', 'name', 'website', 'park_area', 'geometry'], dtype='object')

In [3]:
manhattan_parks_gdf

,id,name,website,park_area,geometry
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8..."
1,relation/2389631,Governors Island,https://www.nps.gov/gois/index.htm,696227.552870,"POLYGON ((583189.67942 4505151.21096, 583176.3..."
2,relation/3690816,Highbridge Park,https://www.nycgovparks.org/parks/highbridge-p...,521310.282769,"MULTIPOLYGON (((590104.73354 4522324.80788, 59..."
3,relation/4099553,Sara D. Roosevelt Park,https://www.nycgovparks.org/parks/sara-d-roose...,36496.652767,"MULTIPOLYGON (((584986.19297 4507940.75482, 58..."
4,relation/5625630,Fort Tryon Park,https://www.nycgovparks.org/parks/fort-tryon-park,281836.601376,"MULTIPOLYGON (((590333.85776 4524352.52505, 59..."
...,...,...,...,...,...
467,node/7447580902,"103 W 62nd St, New York, NY 10023, USA",N/A,313.654849,POINT (585777.32666 4513877.18446)
468,node/7447580929,"625 Jaffe Dr, New York, NY 10023, USA",N/A,313.654849,POINT (585781.01804 4513876.20582)
469,node/7447580898,"103 W 62nd St, New York, NY 10023, USA",N/A,313.654849,POINT (585773.61806 4513871.63477)
470,node/7447580945,"103 W 62nd St, New York, NY 10023, USA",N/A,313.654849,POINT (585779.32539 4513869.31391)


In [4]:
busyness_taxizones= gpd.read_file("yellow_taxi_busyness_details_hourly.geojson")

In [5]:
#busyness_taxizones_sample = busyness_taxizones.sample(n=10000, random_state=42)

In [6]:
#manhattan_parks_gdf= manhattan_parks_gdf.to_crs(epsg=32618)
#busyness_taxizones_sample=busyness_taxizones_sample.to_crs(epsg=32618)

In [7]:
busyness_taxizones=busyness_taxizones.to_crs(epsg=32618)

In [8]:
# Calculate area taxi zones
busyness_taxizones['taxi_zone_area'] = busyness_taxizones.geometry.area

In [9]:
busyness_taxizones.head(5)

,location_id,year,month,day,hour,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,zone,geometry,taxi_zone_area
0,100,2022,1,1,10,5,True,Winter,True,6,Garment District,"MULTIPOLYGON (((585491.231 4511548.637, 585451...",351170.494667
1,100,2022,1,1,11,5,True,Winter,True,9,Garment District,"MULTIPOLYGON (((585491.231 4511548.637, 585451...",351170.494667
2,100,2022,1,1,12,5,True,Winter,True,4,Garment District,"MULTIPOLYGON (((585491.231 4511548.637, 585451...",351170.494667
3,100,2022,1,1,16,5,True,Winter,True,24,Garment District,"MULTIPOLYGON (((585491.231 4511548.637, 585451...",351170.494667
4,100,2022,1,1,17,5,True,Winter,True,4,Garment District,"MULTIPOLYGON (((585491.231 4511548.637, 585451...",351170.494667


In [10]:
manhattan_parks_gdf

,id,name,website,park_area,geometry
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8..."
1,relation/2389631,Governors Island,https://www.nps.gov/gois/index.htm,696227.552870,"POLYGON ((583189.67942 4505151.21096, 583176.3..."
2,relation/3690816,Highbridge Park,https://www.nycgovparks.org/parks/highbridge-p...,521310.282769,"MULTIPOLYGON (((590104.73354 4522324.80788, 59..."
3,relation/4099553,Sara D. Roosevelt Park,https://www.nycgovparks.org/parks/sara-d-roose...,36496.652767,"MULTIPOLYGON (((584986.19297 4507940.75482, 58..."
4,relation/5625630,Fort Tryon Park,https://www.nycgovparks.org/parks/fort-tryon-park,281836.601376,"MULTIPOLYGON (((590333.85776 4524352.52505, 59..."
...,...,...,...,...,...
467,node/7447580902,"103 W 62nd St, New York, NY 10023, USA",N/A,313.654849,POINT (585777.32666 4513877.18446)
468,node/7447580929,"625 Jaffe Dr, New York, NY 10023, USA",N/A,313.654849,POINT (585781.01804 4513876.20582)
469,node/7447580898,"103 W 62nd St, New York, NY 10023, USA",N/A,313.654849,POINT (585773.61806 4513871.63477)
470,node/7447580945,"103 W 62nd St, New York, NY 10023, USA",N/A,313.654849,POINT (585779.32539 4513869.31391)


In [11]:
manhattan_parks_gdf.columns

Index(['id', 'name', 'website', 'park_area', 'geometry'], dtype='object')

In [12]:
busyness_taxizones.columns

Index(['location_id', 'year', 'month', 'day', 'hour', 'day_of_week', 'weekend',
       'season', 'isHoliday?', 'taxi_zone_busyness', 'zone', 'geometry',
       'taxi_zone_area'],
      dtype='object')

In [13]:
busyness_taxizones=busyness_taxizones.rename(columns={'location_id':'taxi_zone_id','passenger_count_number':'taxi_zone_busyness'})

In [14]:
busyness_taxizones=busyness_taxizones.rename(columns={'zone':'taxi_zone'})

In [15]:
manhattan_parks_gdf.columns

Index(['id', 'name', 'website', 'park_area', 'geometry'], dtype='object')

In [16]:
manhattan_parks_gdf=manhattan_parks_gdf.rename(columns={'id':'park_id','name':'park_name'})

#### Spatial join the parks with each taxi zone

In [17]:
# Perform a spatial join to find parks within each taxi zone
parks_with_zones = gpd.sjoin(manhattan_parks_gdf, busyness_taxizones, how='inner', predicate='intersects')

print(parks_with_zones[['park_name', 'taxi_zone_id']])


C:\Users\Justh\AppData\Local\Temp\ipykernel_3960\3175735252.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:32618

  parks_with_zones = gpd.sjoin(manhattan_parks_gdf, busyness_taxizones, how='inner', predicate='intersects')


                                  park_name taxi_zone_id
0                             John Jay Park          140
0                             John Jay Park          140
0                             John Jay Park          140
0                             John Jay Park          140
0                             John Jay Park          140
..                                      ...          ...
471  103 W 62nd St, New York, NY 10023, USA          142
471  103 W 62nd St, New York, NY 10023, USA          142
471  103 W 62nd St, New York, NY 10023, USA          142
471  103 W 62nd St, New York, NY 10023, USA          142
471  103 W 62nd St, New York, NY 10023, USA          142

[895969 rows x 2 columns]


In [18]:
parks_with_zones.head()

,park_id,park_name,website,park_area,geometry,index_right,taxi_zone_id,year,month,day,hour,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone,taxi_zone_area
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17906,140,2022,7,16,0,5,True,Summer,False,16,Lenox Hill East,692556.6829
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17907,140,2022,7,16,1,5,True,Summer,False,2,Lenox Hill East,692556.6829
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17908,140,2022,7,16,11,5,True,Summer,False,4,Lenox Hill East,692556.6829
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17909,140,2022,7,16,12,5,True,Summer,False,9,Lenox Hill East,692556.6829
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17910,140,2022,7,16,19,5,True,Summer,False,11,Lenox Hill East,692556.6829


Calculating remaining taxi zone area

In [19]:
# Calculate the remaining area of the taxi zone
parks_with_zones['remaining_taxi_zone_area'] = parks_with_zones['taxi_zone_area'] - parks_with_zones['park_area']

# Calculate the proportion of the taxi zone area that is covered by the park
parks_with_zones['park_proportion'] = parks_with_zones['park_area'] / parks_with_zones['taxi_zone_area']

# Calculate the busyness score for the park
parks_with_zones['park_busyness'] = parks_with_zones['park_proportion'] * parks_with_zones['taxi_zone_busyness']

# Display the resulting DataFrame
print(parks_with_zones[['park_name', 'park_id','park_area','taxi_zone_area','park_proportion', 'taxi_zone_busyness','park_busyness']])


                                  park_name           park_id    park_area  \
0                             John Jay Park  relation/1751075  9506.859431   
0                             John Jay Park  relation/1751075  9506.859431   
0                             John Jay Park  relation/1751075  9506.859431   
0                             John Jay Park  relation/1751075  9506.859431   
0                             John Jay Park  relation/1751075  9506.859431   
..                                      ...               ...          ...   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   

     taxi_zone_area  park_proportion  taxi_zone_busyness  park_

In [20]:
parks_with_zones.head(20)

,park_id,park_name,website,park_area,geometry,index_right,taxi_zone_id,year,month,day,...,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17906,140,2022,7,16,...,5,True,Summer,False,16,Lenox Hill East,692556.6829,683049.823468,0.013727,0.219635
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17907,140,2022,7,16,...,5,True,Summer,False,2,Lenox Hill East,692556.6829,683049.823468,0.013727,0.027454
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17908,140,2022,7,16,...,5,True,Summer,False,4,Lenox Hill East,692556.6829,683049.823468,0.013727,0.054909
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17909,140,2022,7,16,...,5,True,Summer,False,9,Lenox Hill East,692556.6829,683049.823468,0.013727,0.123545
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17910,140,2022,7,16,...,5,True,Summer,False,11,Lenox Hill East,692556.6829,683049.823468,0.013727,0.150999
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17911,140,2022,7,17,...,6,True,Summer,False,9,Lenox Hill East,692556.6829,683049.823468,0.013727,0.123545
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17912,140,2022,7,17,...,6,True,Summer,False,1,Lenox Hill East,692556.6829,683049.823468,0.013727,0.013727
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17913,140,2022,7,17,...,6,True,Summer,False,10,Lenox Hill East,692556.6829,683049.823468,0.013727,0.137272
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17914,140,2022,7,17,...,6,True,Summer,False,13,Lenox Hill East,692556.6829,683049.823468,0.013727,0.178454
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17915,140,2022,7,18,...,0,False,Summer,False,30,Lenox Hill East,692556.6829,683049.823468,0.013727,0.411816


In [21]:
parks_with_zones.to_file('park_busyness_details_hourly.csv',driver='GeoJSON')

In [22]:
parks_with_zones

,park_id,park_name,website,park_area,geometry,index_right,taxi_zone_id,year,month,day,...,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17906,140,2022,7,16,...,5,True,Summer,False,16,Lenox Hill East,692556.68290,683049.823468,0.013727,0.219635
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17907,140,2022,7,16,...,5,True,Summer,False,2,Lenox Hill East,692556.68290,683049.823468,0.013727,0.027454
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17908,140,2022,7,16,...,5,True,Summer,False,4,Lenox Hill East,692556.68290,683049.823468,0.013727,0.054909
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17909,140,2022,7,16,...,5,True,Summer,False,9,Lenox Hill East,692556.68290,683049.823468,0.013727,0.123545
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((588697.29367 4513735.90726, 588660.8...",17910,140,2022,7,16,...,5,True,Summer,False,11,Lenox Hill East,692556.68290,683049.823468,0.013727,0.150999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,node/7447580924,"103 W 62nd St, New York, NY 10023, USA",N/A,313.654849,POINT (585783.31290 4513866.81769),21110,142,2022,12,14,...,2,False,Winter,False,5,Lincoln Square East,708820.86973,708507.214881,0.000443,0.002213
471,node/7447580924,"103 W 62nd St, New York, NY 10023, USA",N/A,313.654849,POINT (585783.31290 4513866.81769),21109,142,2022,12,14,...,2,False,Winter,False,4,Lincoln Square East,708820.86973,708507.214881,0.000443,0.001770
471,node/7447580924,"103 W 62nd St, New York, NY 10023, USA",N/A,313.654849,POINT (585783.31290 4513866.81769),21108,142,2022,12,14,...,2,False,Winter,False,10,Lincoln Square East,708820.86973,708507.214881,0.000443,0.004425
471,node/7447580924,"103 W 62nd St, New York, NY 10023, USA",N/A,313.654849,POINT (585783.31290 4513866.81769),21107,142,2022,12,14,...,2,False,Winter,False,30,Lincoln Square East,708820.86973,708507.214881,0.000443,0.013275


In [23]:
parks_with_zones.shape

(895969, 21)

In [24]:
#checking for duplicated rows
duplicate_rows = parks_with_zones[parks_with_zones.duplicated()]

#display duplicate rows
print("Duplicate Rows:")
print(duplicate_rows)

Duplicate Rows:
Empty GeoDataFrame
Columns: [park_id, park_name, website, park_area, geometry, index_right, taxi_zone_id, year, month, day, hour, day_of_week, weekend, season, isHoliday?, taxi_zone_busyness, taxi_zone, taxi_zone_area, remaining_taxi_zone_area, park_proportion, park_busyness]
Index: []

[0 rows x 21 columns]


In [25]:
park_busyness_details= parks_with_zones.groupby(['park_id','park_name','taxi_zone_id','geometry'])['park_busyness'].sum().reset_index()


In [26]:
type(park_busyness_details)

pandas.core.frame.DataFrame

In [27]:
# Set the geometry column explicitly
park_busyness_details= gpd.GeoDataFrame(park_busyness_details, geometry='geometry')

In [28]:
park_busyness_details

,park_id,park_name,taxi_zone_id,geometry,park_busyness
0,node/11457630561,Kowsky Plaza,13,POINT (583037.17345 4507235.73811),8.768645
1,node/1203709404,Sherman Creek,127,POINT (591160.55027 4523886.46508),0.271311
2,node/3369582883,"103 W 62nd St, New York, NY 10023, USA",142,POINT (585777.52241 4513869.75931),19.147074
3,node/3369582884,"625 Jaffe Dr, New York, NY 10023, USA",142,POINT (585780.48512 4513881.40660),19.147074
4,node/3369582885,"103 W 62nd St, New York, NY 10023, USA",142,POINT (585782.07805 4513872.86522),19.147074
...,...,...,...,...,...
495,way/973521995,Ahearn Park,232,"POLYGON ((585841.66647 4507605.11797, 585842.7...",1.993613
496,way/976724943,"1120 Cooper Sq, New York, NY 10003, USA",113,"POLYGON ((585192.35882 4509213.93738, 585188.2...",6.923207
497,way/985281628,Seward Park High School - Community Playground,232,"POLYGON ((585341.64492 4507674.79881, 585395.6...",22.454404
498,way/990936691,Pier 42 Upland Park,232,"POLYGON ((586058.72419 4507144.47324, 586025.7...",94.466446


In [29]:
park_busyness_details.geom_type.value_counts()

Polygon         420
Point            40
MultiPolygon     33
LineString        7
Name: count, dtype: int64

In [30]:
park_busyness_details.explore("park_busyness", legend=True)

In [31]:
central_park_details= parks_with_zones[parks_with_zones['park_name']=='Central Park']
central_park_details

,park_id,park_name,website,park_area,geometry,index_right,taxi_zone_id,year,month,day,...,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness
279,way/427818536,Central Park,https://www.centralparknyc.org,3.413739e+06,"POLYGON ((587923.35814 4517112.63363, 587921.5...",96422,43,2022,9,20,...,1,False,Autumn,False,6,Central Park,3.556719e+06,142980.04328,0.9598,5.758800
279,way/427818536,Central Park,https://www.centralparknyc.org,3.413739e+06,"POLYGON ((587923.35814 4517112.63363, 587921.5...",96423,43,2022,9,20,...,1,False,Autumn,False,5,Central Park,3.556719e+06,142980.04328,0.9598,4.799000
279,way/427818536,Central Park,https://www.centralparknyc.org,3.413739e+06,"POLYGON ((587923.35814 4517112.63363, 587921.5...",96424,43,2022,9,20,...,1,False,Autumn,False,29,Central Park,3.556719e+06,142980.04328,0.9598,27.834201
279,way/427818536,Central Park,https://www.centralparknyc.org,3.413739e+06,"POLYGON ((587923.35814 4517112.63363, 587921.5...",96425,43,2022,9,20,...,1,False,Autumn,False,2,Central Park,3.556719e+06,142980.04328,0.9598,1.919600
279,way/427818536,Central Park,https://www.centralparknyc.org,3.413739e+06,"POLYGON ((587923.35814 4517112.63363, 587921.5...",96426,43,2022,9,21,...,2,False,Autumn,False,23,Central Park,3.556719e+06,142980.04328,0.9598,22.075400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,way/427818536,Central Park,https://www.centralparknyc.org,3.413739e+06,"POLYGON ((587923.35814 4517112.63363, 587921.5...",96005,43,2022,6,26,...,6,True,Summer,False,28,Central Park,3.556719e+06,142980.04328,0.9598,26.874401
279,way/427818536,Central Park,https://www.centralparknyc.org,3.413739e+06,"POLYGON ((587923.35814 4517112.63363, 587921.5...",96006,43,2022,6,26,...,6,True,Summer,False,3,Central Park,3.556719e+06,142980.04328,0.9598,2.879400
279,way/427818536,Central Park,https://www.centralparknyc.org,3.413739e+06,"POLYGON ((587923.35814 4517112.63363, 587921.5...",96007,43,2022,6,27,...,0,False,Summer,False,5,Central Park,3.556719e+06,142980.04328,0.9598,4.799000
279,way/427818536,Central Park,https://www.centralparknyc.org,3.413739e+06,"POLYGON ((587923.35814 4517112.63363, 587921.5...",94811,43,2022,10,27,...,3,False,Autumn,False,16,Central Park,3.556719e+06,142980.04328,0.9598,15.356800
